In [22]:
import sys
sys.path.append('../zoho_data_app')

import main_module as m

import pandas as pd
import gspread
import re
import datetime

import dateutil

%load_ext autoreload
%autoreload 2


# file_loc = r'C:\Users\fajar\Documents\Python\Data'
file_loc = r'C:\Users\fajar\Documents\Python\Data'

gc = gspread.oauth(
    credentials_filename= r'C:\Users\fajar\Documents\Python\Data\Google Credentials\karma-oauth.json'
)

SHEET_ID_SOURCE = '1PiRLPCroErrgZ07dlLdZZxUKzFmSZNnFwLy48MDmK2A'
execeptional_list = ['Database Name', 'Sheet Name', 'Reciprocal']


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
# Open the spreadsheet
spreadsheet_source = gc.open_by_key(SHEET_ID_SOURCE)

# Get all sheet names
sheet_names = [sheet.title for sheet in spreadsheet_source.worksheets()]
sheet_names_id = [sheet.id for sheet in spreadsheet_source.worksheets()]
print(sheet_names)

['Database Name', 'Sheet Name', 'HOK', 'KC 2021', 'KCB-St.Martin', 'KCB-Elite-ENG', 'KCB-ID-Excluded KCID', 'KCB-NH-ID', 'KCB-NH-ENG', 'KCB-ENG-Excluded KCID', 'KCE', 'KCID-ID', 'KCID-ENG', 'KCI-Elite-Non India', 'KCI-India', 'KCI-NH-India', 'KCI-NH-Non India', 'Reciprocal', 'KCI-Non India']


In [24]:
len(sheet_names)

19

In [25]:

count = 0
frames = []

while count < len(sheet_names):
    if sheet_names[count] not in execeptional_list:
        print(f"Use this list {sheet_names[count]}")
        sheet = spreadsheet_source.worksheet(sheet_names[count])

        globals()['worksheet%s' % count] = spreadsheet_source.worksheet(sheet_names[count])
        globals()['rows%s' % count] = globals()['worksheet%s' % count].get_all_records()

        globals()['df%s' % count] = pd.DataFrame(globals()['rows%s' % count])

        globals()['df%s' % count]['Sheet Name'] = sheet_names[count]

        frames.append(globals()['df%s' % count])
        # print(f"Sheet {sheet_names[count]} has {sheet.row_count} rows and {sheet.col_count} columns")
        count += 1
    else:
        print(f"Not Use This List {sheet_names[count]}")

        count += 1
    
result = pd.concat(frames, ignore_index=True)
result
                    

Not Use This List Database Name
Not Use This List Sheet Name
Use this list HOK
Use this list KC 2021
Use this list KCB-St.Martin
Use this list KCB-Elite-ENG
Use this list KCB-ID-Excluded KCID
Use this list KCB-NH-ID
Use this list KCB-NH-ENG
Use this list KCB-ENG-Excluded KCID
Use this list KCE
Use this list KCID-ID
Use this list KCID-ENG
Use this list KCI-Elite-Non India
Use this list KCI-India
Use this list KCI-NH-India
Use this list KCI-NH-Non India
Not Use This List Reciprocal
Use this list KCI-Non India


,number,ContactID,FirstName,LastName,AddressLine1,AddressLine2,Town,City,Country,Post Code,...,WeddingDate,Keywords,PanCardNo,Account Type,Account Status,OdysseyNo,Servicing Office,Title Database,Sheet Name,Account Currency
0,1118668,1109047,Elliot,Styche,,,Sydney,,Australia,,...,,,,HOUSE OF KARMA,Active,,KARMA CONCIERGE BALI,HOK,HOK,NaN
1,1118840,1109358,Marcus,Willis,"8/79 Oxford St,",COLLINGWOOD,MELBOURNE,VICTORIA,Australia,3066,...,,,,HOUSE OF KARMA,Active,,KARMA CONCIERGE BALI,HOK,HOK,NaN
2,1118843,1109361,Jackson,Springer,,,MELBOURNE,VICTORIA,Australia,,...,,,,HOUSE OF KARMA,Active,,KARMA CONCIERGE BALI,HOK,HOK,NaN
3,1118885,1109425,Kirk,Mathews,,,SYDNEY,NEW SOUTH WALES,Australia,,...,,,,HOUSE OF KARMA,Active,,KARMA CONCIERGE BALI,HOK,HOK,NaN
4,1118886,1109427,Alexander,Parr,,,SYDNEY,NEW SOUTH WALES,Australia,,...,,,,HOUSE OF KARMA,Active,,KARMA CONCIERGE BALI,HOK,HOK,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78797,998930,919671,Hazel,Lynes,BROCKSDALE FARM,Welcombe,WELCOMBE,DEVON,England,EX39 6HE,...,,,,KARMA CLUB,Active,C82046030,KARMA CONCIERGE INDIA,NaN,KCI-Non India,USD
78798,999064,919883,William Jon,Cushing,SAUCEFIELDS FARM,"BACK O ""TH"" BROOK WATERFALL",WATERHOUSES,STAFFORDSHIRE,England,ST1U 3JA,...,,,,KARMA CLUB,Active,C82051647,KARMA CONCIERGE INDIA,NaN,KCI-Non India,USD
78799,999552,920580,Christopher Richard,Gill,PARKHILL COTTAGE,BRITTAINS LANE,SEVENOAKS,KENT,England,TN13 2NG,...,,,,KARMA CLUB,Active,C82051493,KARMA CONCIERGE INDIA,NaN,KCI-Non India,USD
78800,132366,15009,Ann,Gilchrist,HAWBERRY HOUSE,CLARENDON ROAD,MANCHESTER,MANCHESTER,England,M34,...,,Special Waiver,,,Active,C82045914,KARMA CONCIERGE INDIA,NaN,KCI-Non India,


In [26]:
result.columns

Index(['number', 'ContactID', 'FirstName', 'LastName', 'AddressLine1',
       'AddressLine2', 'Town', 'City', 'Country', 'Post Code', 'Email',
       'Phones', 'Owner', 'Primary', 'Family', 'Nationality', 'DoB',
       'WeddingDate', 'Keywords', 'PanCardNo', 'Account Type',
       'Account Status', 'OdysseyNo', 'Servicing Office', 'Title Database',
       'Sheet Name', 'Account Currency'],
      dtype='object')

In [27]:
result['Main DB'] = ''
result['Sub DB'] = ''

result.loc[result['Sheet Name'] == 'HOK', 'Main DB'] = 'HOK'
result.loc[result['Sheet Name'] == 'KC 2021', ['Main DB','Sub DB']] = ['KC 2021','KCEU']
result.loc[result['Sheet Name'] == 'KCB-Elite-ENG', ['Main DB','Sub DB']] = ['KCB', 'Elite-Eng']
result.loc[result['Sheet Name'] == 'KCB-ID-Excluded KCID', ['Main DB','Sub DB']] = ['KCB', 'ID-Exc']
result.loc[result['Sheet Name'] == 'KCB-NH-ID', ['Main DB','Sub DB']] = ['KCB', 'ID-NH']
result.loc[result['Sheet Name'] == 'KCB-NH-ENG', ['Main DB','Sub DB']] = ['KCB', 'ENG-NH']
result.loc[result['Sheet Name'] == 'KCB-ENG-Excluded KCID', ['Main DB','Sub DB']] = ['KCB', 'ENG-Exc']
result.loc[result['Sheet Name'] == 'KCB-St.Martin', ['Main DB','Sub DB']] = ['KCB', 'ST.Martin-Elite']
result.loc[result['Sheet Name'] == 'KCE', ['Main DB']] = ['KCDEU']
result.loc[result['Sheet Name'] == 'KCI-Elite-Non India', ['Main DB','Sub DB']] = ['KCI', 'Elite-Non India']
result.loc[result['Sheet Name'] == 'KCI-India', ['Main DB','Sub DB']] = ['KCI', 'India']
result.loc[result['Sheet Name'] == 'KCI-NH-India', ['Main DB','Sub DB']] = ['KCI', 'NH-India']
result.loc[result['Sheet Name'] == 'KCI-NH-Non India', ['Main DB','Sub DB']] = ['KCI', 'NH-Non India']
result.loc[result['Sheet Name'] == 'KCI-Non India', ['Main DB','Sub DB']] = ['KCI', 'Non India']
result.loc[result['Sheet Name'] == 'KCID-ID', ['Main DB','Sub DB']] = ['KCID', 'ID']
result.loc[result['Sheet Name'] == 'KCID-ENG', ['Main DB','Sub DB']] = ['KCID', 'ENG']

In [28]:
current_date = datetime.date.today()

formatted_date = current_date.strftime("%d-%m-%Y")
result['Date DB'] = formatted_date

result['Tags'] = 'Members'

In [29]:
# if data dated is string
# using error=cource to prevent N/A value errors
result['DoB'] = pd.to_datetime(result['DoB'], format='mixed', errors='coerce')
result['DoB'] = result['DoB'].dt.strftime("%d-%m-%Y")

result['WeddingDate'] = pd.to_datetime(result['WeddingDate'], format='mixed', errors='coerce')
result['WeddingDate'] = result['WeddingDate'].dt.strftime("%d-%m-%Y")

In [30]:
result['DoB']

0        12-07-1988
1        20-12-1989
2        14-07-2023
3        30-03-2023
4        13-08-1992
            ...    
78797    06-05-1969
78798    01-01-1900
78799    28-12-1965
78800           NaN
78801           NaN
Name: DoB, Length: 78802, dtype: object

In [31]:
# result.columns
# result.columns = ['number', 'ContactID', 'FirstName', 'LastName', 'AddressLine1',
#        'AddressLine2', 'Town', 'City', 'Country', 'Post Code', 'Email',
#        'Phones', 'Owner', 'Primary', 'Family', 'Nationality', 'DoB',
#        'WeddingDate', 'Account Type', 'Account Status', 'Servicing Office', 
#        'Keywords', 'PanCardNo', 'OdysseyNo', 'segment',
#        'Account Currency', 'Main DB', 'Sub DB', 'Date DB', 'Tags']

In [32]:
result.rename(columns={
    'segment': 'Member Segment',
}, inplace=True)

In [33]:
result.fillna('', inplace=True)

In [34]:
result['Email Old'] = result['Email']

m.clean_space(result, 'Email')
m.email_correction(result, 'Email')

result['Email Correction'] = 'No'
result.loc[result['Email Old']!=result['Email'], 'Email Correction'] = 'Yes'

c:\Users\fajar\Documents\Python\Task Offline\member_app\../zoho_data_app\main_module.py:90: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)
c:\Users\fajar\Documents\Python\Task Offline\member_app\../zoho_data_app\main_module.py:193: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on whi

In [35]:
result[['Email','Main DB']].loc[result['Email'].str.contains(',', na=False)]

,Email,Main DB
6074,"neil.aitchison@karmagroup.com,447947236783,",KCB


In [36]:
result.columns

Index(['number', 'ContactID', 'FirstName', 'LastName', 'AddressLine1',
       'AddressLine2', 'Town', 'City', 'Country', 'Post Code', 'Email',
       'Phones', 'Owner', 'Primary', 'Family', 'Nationality', 'DoB',
       'WeddingDate', 'Keywords', 'PanCardNo', 'Account Type',
       'Account Status', 'OdysseyNo', 'Servicing Office', 'Title Database',
       'Sheet Name', 'Account Currency', 'Main DB', 'Sub DB', 'Date DB',
       'Tags', 'Email Old', 'Email Correction'],
      dtype='object')

In [37]:
result = result.reindex(columns=['number', 'ContactID', 'FirstName', 'LastName', 'AddressLine1',
       'AddressLine2', 'Town', 'City', 'Country', 'Post Code', 'Email Old','Email',
       'Email Correction','Phones', 'Owner', 'Primary', 'Family', 'Nationality', 'DoB',
       'WeddingDate', 'Account Type', 'Account Status', 'Servicing Office',
       'Sheet Name', 'Keywords', 'PanCardNo', 'OdysseyNo', 'Member Segment',
       'Account Currency', 'Main DB', 'Sub DB', 'Date DB', 'Tags'])

In [38]:
result.rename(columns={
    'Town': 'State'
    # Add other renames as needed...
}, inplace=True)

In [44]:
result.to_excel(file_loc+'\compiled-member.xlsx', index=False)
# result.to_excel(file_loc+'\compiled-member-test.xlsx', index=False)

In [40]:
result = result.reindex(columns=['number', 'ContactID', 'FirstName', 'LastName', 'AddressLine1',
       'AddressLine2', 'State', 'City', 'Country', 'Post Code','Email',
       'Email Correction','Phones', 'Owner', 'Primary', 'Family', 'Nationality', 'DoB',
       'WeddingDate', 'Account Type', 'Account Status', 'Servicing Office', 'Keywords', 'PanCardNo', 'OdysseyNo', 'Member Segment',
       'Account Currency', 'Main DB', 'Sub DB', 'Date DB', 'Tags'])

In [41]:
result.to_csv(file_loc+'\compiled-member.csv', index=False)
# result.to_csv(file_loc+'\compiled-member-test.csv', index=False)

In [42]:
# result.loc[result['Email'] == 'elliotstyche@hotmail.co.uk'].to_csv(r'C:\Users\fajar\Documents\Python\Data\test-upload.csv', index=False)